In [2]:
from gensim.models import word2vec
import MeCab
import numpy as np
import json
import hashlib
from sklearn.cross_decomposition import CCA
from googletrans import Translator
import numpy as np
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import random

# 文章ベクトルの学習フェーズ

In [3]:
translator = Translator()
mt = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
mt.parse('')

'EOS\n'

In [4]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
model_doc = Doc2Vec.load("jawiki.doc2vec.dbow300d.model")
model_word = word2vec.Word2Vec.load("wiki_plus.model")

In [5]:
def get_tags(text):
    word = {}
    node = mt.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        if (fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞') and node.surface in model_word.wv:
            w = node.surface
            word[w] = word.get(w, 0) + 1
        node = node.next
    return word
def weighted_mean_vec(text):
    v = np.zeros(model_word.vector_size)
    s = 1.0
    for w,weight in get_tags(text).items():
        v += weight * model_word.wv[w]  #Eventクラスeの単語wの個数＊単語wのベクトル
        s += weight
    return v / s

In [6]:
def get_tags_for_doc2vec(text):
    word = []
    node = mt.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        if node.surface in model_doc.wv:
            w = node.surface
            word.append(w)
        node = node.next
    return word

In [7]:
import pandas as pd
df = pd.read_excel('データ/ひろしま観光ナビ.xlsx')
df = df.drop('Unnamed: 0', axis=1)
df

,観光名所,桜名所,テーマパーク,スポーツ施設,体験・見学,自然・景勝地,サイクリング,美術館・博物館,道の駅,安全安心対策,観光案内所,世界遺産,ビーチ・プール,公園・キャンプ場,お土産・カフェ等,紅葉名所,スキー,クルーズ・船,フラワー,温泉・癒し・スパ
0,今高野山,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,土師ダム（八千代湖）,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,縮景園,1,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,1,0
3,錦帯橋,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0
4,平和記念公園,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1747,香遊温泉,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1748,養老温泉(尾道市),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1749,夢ヶ丘温泉湯めガーデン,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1750,スーパー銭湯ゆ～ゆ～えびす,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [8]:
#観光名所を全てベクトル化(word2vec)
columns_vector_word ={}
for i in df:
    if i == '観光名所':
            continue
    else:
        columns_vector_word[i]= weighted_mean_vec(i)

In [9]:
#観光名所を全てベクトル化
columns_vector_doc = {}
for i in df:
    if i == '観光名所':
            continue
    else:
        columns_vector_doc[i]= model_doc.infer_vector(get_tags_for_doc2vec(i))

# データを (観光名所の文章ベクトル,カテゴリーの文章ベクトル)
# ラベルを関連ある時は１,ない時は0とする.

In [10]:
class Label:
    ARI = 1
    NASI = 0
data_word = []
data_dict_word = {}
labels_word = []
for index, row in df.iterrows():
    x1 = weighted_mean_vec(row['観光名所'])
    for i in df:
        if i == '観光名所':
            continue
        else:
            x2 = columns_vector_word[i]
            data_word.extend([np.append(x1, x2)])
            data_dict_word[tuple(np.append(x1, x2))] = row['観光名所']+' : '+i
            if row[i] == 1:
                labels_word.append(Label.ARI)
            else:
                labels_word.append(Label.NASI)

In [11]:
class Label:
    ARI = 1
    NASI = 0
data_doc = []
data_dict_doc = {}
labels_doc = []
for index, row in df.iterrows():
    x1 = model_doc.infer_vector(get_tags_for_doc2vec(row['観光名所']))
    for i in df:
        if i == '観光名所':
            continue
        else:
            x2 = columns_vector_doc[i]
            data_doc.extend([np.append(x1, x2)])
            data_dict_doc[tuple(np.append(x1, x2))] = row['観光名所']+' : '+i
            if row[i] == 1:
                labels_doc.append(Label.ARI)
            else:
                labels_doc.append(Label.NASI)

In [12]:
def train_data(data,labels):
    index_1 = [i for i, x in enumerate(labels) if x == 1]
    index_0 = [i for i, x in enumerate(labels) if x == 0]
    index_0 = random.sample(index_0, len(index_1))
    data_1 = [data[i] for i in index_1]
    data_0 = [data[i] for i in index_0]
    labels = [Label.ARI]*len(data_1) + [Label.NASI]*len(data_0) 
    data = data_1 + data_0
    data = np.array(data)
    labels = np.array(labels)
    return data, labels

In [13]:
data_word, labels_word = train_data(data_word,labels_word)
data_doc, labels_doc = train_data(data_doc,labels_doc)

In [14]:
print('data:',data_word)
print('labels:',labels_word)
print('labelの１の数:',labels_word.sum())

data: [[ 0.87101964 -0.88109796 -0.63586656 ... -2.85154661 -0.94351971
  -1.27005013]
 [ 0.87101964 -0.88109796 -0.63586656 ... -2.87626696 -0.81296885
  -1.63384255]
 [-0.09097033 -0.03834292 -0.17624676 ... -2.85154661 -0.94351971
  -1.27005013]
 ...
 [-0.89617664  0.31462714  0.07573295 ...  0.08759923  0.19916581
   0.01619013]
 [-1.33980238 -0.51458855 -2.33965325 ... -0.98338791 -0.57814906
  -0.56309945]
 [ 0.43055393 -1.2646328  -1.71746745 ... -1.12513877 -0.03159237
  -1.42743503]]
labels: [1 1 1 ... 0 0 0]
labelの１の数: 2730


In [15]:
print('data_word.shape', data_word.shape,'     data_doc.shape',data_doc.shape)
print('labels_word.shape', labels_word.shape,'          labels_doc.shape',labels_doc.shape)

data_word.shape (5460, 400)      data_doc.shape (5460, 600)
labels_word.shape (5460,)           labels_doc.shape (5460,)


In [16]:
def gakusyu(data,labels):
    x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)
    print('トレーニングデータの全体の数:',x_train.shape[0])
    print('トレーニングデータの1の数    :',y_train.sum())
    print('トレーニングデータ1の割合    :',round(y_train.sum()/x_train.shape[0]*100,1),'%')
    print('テストデータの全体の数         :',x_test.shape[0])
    print('テストデータの１の数            :',y_test.sum())
    print('テストデータの1の割合          :',round(y_test.sum()/x_test.shape[0]*100,1),'%')
    # y_train を one-hot 表現にする。
    y_train_one_hot = np_utils.to_categorical(y_train)

    # モデルを作成する。
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(data.shape[1],))) #activation
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))

    model.compile(
        optimizer='adam', #最適化アルゴリズムの種類.sgd,rmsprop,adam,adagrad,nadam,adadeltaなどある。
        loss='categorical_crossentropy', #損失関数名
        metrics=['accuracy']) #訓練やテストの際にモデルを評価するための評価関数のリスト． 典型的には metrics=['accuracy']を使用する.

    # 学習を実行する。
    model.fit(x_train, y_train_one_hot , batch_size=16, epochs=30) #epochs 学習回数,batch_size 重みを調整の際に行う平均化のときに使うデータの数（dataの数/batch_size回）×epochs

    # 推論する。
    prob = model.predict(x_test, batch_size=len(x_test))

    # 最も確率が高いのを推定クラスとする。
    y_pred = np.argmax(prob, axis=1)

    # 精度を検証する。
    accuracy = accuracy_score(y_test, y_pred)
    print('accuracy: {:.2%}'.format(accuracy))
    return accuracy, model, x_test, y_test

In [17]:
history = {'word2vec':0,'doc2vec':0,'w_count':0,'d_count':0}

In [18]:
for i in range(10):
    data_word, labels_word = train_data(data_word,labels_word)
    accuracy, model, x_test, y_test = gakusyu(data_word, labels_word)
    history['word2vec'] = history['word2vec'] + accuracy
    history['w_count'] += 1

トレーニングデータの全体の数: 4368
トレーニングデータの1の数    : 2203
トレーニングデータ1の割合    : 50.4 %
テストデータの全体の数         : 1092
テストデータの１の数            : 527
テストデータの1の割合          : 48.3 %
Epoch 1/30
273/273 [==============================] - 0s 758us/step - loss: 0.5491 - accuracy: 0.7347
Epoch 2/30
273/273 [==============================] - 0s 710us/step - loss: 0.4616 - accuracy: 0.7859
Epoch 3/30
273/273 [==============================] - 0s 672us/step - loss: 0.4202 - accuracy: 0.8139
Epoch 4/30
273/273 [==============================] - 0s 623us/step - loss: 0.3833 - accuracy: 0.8285
Epoch 5/30
273/273 [==============================] - 0s 606us/step - loss: 0.3684 - accuracy: 0.8331
Epoch 6/30
273/273 [==============================] - 0s 621us/step - loss: 0.3502 - accuracy: 0.8459
Epoch 7/30
273/273 [==============================] - 0s 622us/step - loss: 0.3311 - accuracy: 0.8521
Epoch 8/30
273/273 [==============================] - 0s 693us/step - loss: 0.3241 - accuracy: 0.8498
Epoch 9/30
273/273 [========

273/273 [==============================] - 0s 627us/step - loss: 0.2637 - accuracy: 0.8862
Epoch 16/30
273/273 [==============================] - 0s 610us/step - loss: 0.2420 - accuracy: 0.8915
Epoch 17/30
273/273 [==============================] - 0s 619us/step - loss: 0.2421 - accuracy: 0.8938
Epoch 18/30
273/273 [==============================] - 0s 639us/step - loss: 0.2423 - accuracy: 0.8963
Epoch 19/30
273/273 [==============================] - 0s 628us/step - loss: 0.2224 - accuracy: 0.9018
Epoch 20/30
273/273 [==============================] - 0s 629us/step - loss: 0.2205 - accuracy: 0.9050
Epoch 21/30
273/273 [==============================] - 0s 611us/step - loss: 0.2140 - accuracy: 0.9032
Epoch 22/30
273/273 [==============================] - 0s 609us/step - loss: 0.2175 - accuracy: 0.9036
Epoch 23/30
273/273 [==============================] - 0s 609us/step - loss: 0.2075 - accuracy: 0.9075
Epoch 24/30
273/273 [==============================] - 0s 625us/step - loss: 0.2113 -

accuracy: 81.04%
トレーニングデータの全体の数: 4368
トレーニングデータの1の数    : 2184
トレーニングデータ1の割合    : 50.0 %
テストデータの全体の数         : 1092
テストデータの１の数            : 546
テストデータの1の割合          : 50.0 %
Epoch 1/30
273/273 [==============================] - 0s 618us/step - loss: 0.5539 - accuracy: 0.7257
Epoch 2/30
273/273 [==============================] - 0s 621us/step - loss: 0.4529 - accuracy: 0.7917
Epoch 3/30
273/273 [==============================] - 0s 630us/step - loss: 0.4045 - accuracy: 0.8168
Epoch 4/30
273/273 [==============================] - 0s 602us/step - loss: 0.3835 - accuracy: 0.8276
Epoch 5/30
273/273 [==============================] - 0s 606us/step - loss: 0.3625 - accuracy: 0.8402
Epoch 6/30
273/273 [==============================] - 0s 610us/step - loss: 0.3460 - accuracy: 0.8473
Epoch 7/30
273/273 [==============================] - 0s 604us/step - loss: 0.3209 - accuracy: 0.8516
Epoch 8/30
273/273 [==============================] - 0s 604us/step - loss: 0.3059 - accuracy: 0.8668
Epoch 9/30


273/273 [==============================] - 0s 697us/step - loss: 0.5745 - accuracy: 0.7212
Epoch 2/30
273/273 [==============================] - 0s 679us/step - loss: 0.4628 - accuracy: 0.7880
Epoch 3/30
273/273 [==============================] - 0s 664us/step - loss: 0.4292 - accuracy: 0.8031
Epoch 4/30
273/273 [==============================] - 0s 713us/step - loss: 0.3953 - accuracy: 0.8203
Epoch 5/30
273/273 [==============================] - 0s 649us/step - loss: 0.3703 - accuracy: 0.8320
Epoch 6/30
273/273 [==============================] - 0s 629us/step - loss: 0.3534 - accuracy: 0.8432
Epoch 7/30
273/273 [==============================] - 0s 724us/step - loss: 0.3359 - accuracy: 0.8464
Epoch 8/30
273/273 [==============================] - 0s 735us/step - loss: 0.3225 - accuracy: 0.8583
Epoch 9/30
273/273 [==============================] - 0s 765us/step - loss: 0.3184 - accuracy: 0.8617
Epoch 10/30
273/273 [==============================] - 0s 771us/step - loss: 0.2981 - accurac

273/273 [==============================] - 0s 665us/step - loss: 0.4616 - accuracy: 0.7843
Epoch 3/30
273/273 [==============================] - 0s 622us/step - loss: 0.4228 - accuracy: 0.8017
Epoch 4/30
273/273 [==============================] - 0s 594us/step - loss: 0.3923 - accuracy: 0.8283
Epoch 5/30
273/273 [==============================] - 0s 596us/step - loss: 0.3728 - accuracy: 0.8299
Epoch 6/30
273/273 [==============================] - 0s 599us/step - loss: 0.3586 - accuracy: 0.8363
Epoch 7/30
273/273 [==============================] - 0s 597us/step - loss: 0.3374 - accuracy: 0.8491
Epoch 8/30
273/273 [==============================] - 0s 597us/step - loss: 0.3193 - accuracy: 0.8558
Epoch 9/30
273/273 [==============================] - 0s 601us/step - loss: 0.3109 - accuracy: 0.8652
Epoch 10/30
273/273 [==============================] - 0s 611us/step - loss: 0.2942 - accuracy: 0.8723
Epoch 11/30
273/273 [==============================] - 0s 597us/step - loss: 0.2855 - accura

In [19]:
for i in range(10):
    data_doc, labels_doc = train_data(data_doc,labels_doc)
    accuracy, model,x_test, y_test = gakusyu(data_doc, labels_doc)
    history['doc2vec'] = history['doc2vec'] + accuracy
    history['d_count'] += 1

トレーニングデータの全体の数: 4368
トレーニングデータの1の数    : 2167
トレーニングデータ1の割合    : 49.6 %
テストデータの全体の数         : 1092
テストデータの１の数            : 563
テストデータの1の割合          : 51.6 %
Epoch 1/30
273/273 [==============================] - 0s 688us/step - loss: 0.6033 - accuracy: 0.6603
Epoch 2/30
273/273 [==============================] - 0s 690us/step - loss: 0.5399 - accuracy: 0.7241
Epoch 3/30
273/273 [==============================] - 0s 683us/step - loss: 0.5165 - accuracy: 0.7374
Epoch 4/30
273/273 [==============================] - 0s 674us/step - loss: 0.4788 - accuracy: 0.7553
Epoch 5/30
273/273 [==============================] - 0s 683us/step - loss: 0.4397 - accuracy: 0.7905
Epoch 6/30
273/273 [==============================] - 0s 669us/step - loss: 0.4139 - accuracy: 0.8134
Epoch 7/30
273/273 [==============================] - 0s 678us/step - loss: 0.3882 - accuracy: 0.8246
Epoch 8/30
273/273 [==============================] - 0s 675us/step - loss: 0.3690 - accuracy: 0.8320
Epoch 9/30
273/273 [========

273/273 [==============================] - 0s 677us/step - loss: 0.5869 - accuracy: 0.6886
Epoch 2/30
273/273 [==============================] - 0s 668us/step - loss: 0.5342 - accuracy: 0.7237
Epoch 3/30
273/273 [==============================] - 0s 666us/step - loss: 0.5061 - accuracy: 0.7477
Epoch 4/30
273/273 [==============================] - 0s 663us/step - loss: 0.4775 - accuracy: 0.7706
Epoch 5/30
273/273 [==============================] - 0s 659us/step - loss: 0.4398 - accuracy: 0.7905
Epoch 6/30
273/273 [==============================] - 0s 654us/step - loss: 0.4079 - accuracy: 0.8123
Epoch 7/30
273/273 [==============================] - 0s 659us/step - loss: 0.3808 - accuracy: 0.8304
Epoch 8/30
273/273 [==============================] - 0s 668us/step - loss: 0.3531 - accuracy: 0.8496
Epoch 9/30
273/273 [==============================] - 0s 658us/step - loss: 0.3362 - accuracy: 0.8503
Epoch 10/30
273/273 [==============================] - 0s 660us/step - loss: 0.3268 - accurac

273/273 [==============================] - 0s 681us/step - loss: 0.5265 - accuracy: 0.7299
Epoch 3/30
273/273 [==============================] - 0s 655us/step - loss: 0.5013 - accuracy: 0.7518
Epoch 4/30
273/273 [==============================] - 0s 657us/step - loss: 0.4554 - accuracy: 0.7880
Epoch 5/30
273/273 [==============================] - 0s 654us/step - loss: 0.4313 - accuracy: 0.7990
Epoch 6/30
273/273 [==============================] - 0s 668us/step - loss: 0.3917 - accuracy: 0.8265
Epoch 7/30
273/273 [==============================] - 0s 667us/step - loss: 0.3716 - accuracy: 0.8292
Epoch 8/30
273/273 [==============================] - 0s 669us/step - loss: 0.3673 - accuracy: 0.8361
Epoch 9/30
273/273 [==============================] - 0s 662us/step - loss: 0.3415 - accuracy: 0.8503
Epoch 10/30
273/273 [==============================] - 0s 694us/step - loss: 0.3250 - accuracy: 0.8601
Epoch 11/30
273/273 [==============================] - 0s 669us/step - loss: 0.3124 - accura

273/273 [==============================] - 0s 677us/step - loss: 0.5085 - accuracy: 0.7477
Epoch 4/30
273/273 [==============================] - 0s 667us/step - loss: 0.4593 - accuracy: 0.7809
Epoch 5/30
273/273 [==============================] - 0s 673us/step - loss: 0.4372 - accuracy: 0.8015
Epoch 6/30
273/273 [==============================] - 0s 674us/step - loss: 0.4093 - accuracy: 0.8194
Epoch 7/30
273/273 [==============================] - 0s 663us/step - loss: 0.3888 - accuracy: 0.8278
Epoch 8/30
273/273 [==============================] - 0s 664us/step - loss: 0.3735 - accuracy: 0.8317
Epoch 9/30
273/273 [==============================] - 0s 669us/step - loss: 0.3506 - accuracy: 0.8484
Epoch 10/30
273/273 [==============================] - 0s 666us/step - loss: 0.3393 - accuracy: 0.8549
Epoch 11/30
273/273 [==============================] - 0s 683us/step - loss: 0.3210 - accuracy: 0.8594
Epoch 12/30
273/273 [==============================] - 0s 677us/step - loss: 0.3106 - accur

273/273 [==============================] - 0s 675us/step - loss: 0.4633 - accuracy: 0.7717
Epoch 5/30
273/273 [==============================] - 0s 695us/step - loss: 0.4350 - accuracy: 0.7974
Epoch 6/30
273/273 [==============================] - 0s 681us/step - loss: 0.4024 - accuracy: 0.8132
Epoch 7/30
273/273 [==============================] - 0s 682us/step - loss: 0.3783 - accuracy: 0.8283
Epoch 8/30
273/273 [==============================] - 0s 674us/step - loss: 0.3561 - accuracy: 0.8388
Epoch 9/30
273/273 [==============================] - 0s 650us/step - loss: 0.3453 - accuracy: 0.8466
Epoch 10/30
273/273 [==============================] - 0s 658us/step - loss: 0.3213 - accuracy: 0.8615
Epoch 11/30
273/273 [==============================] - 0s 663us/step - loss: 0.3027 - accuracy: 0.8702
Epoch 12/30
273/273 [==============================] - 0s 675us/step - loss: 0.2909 - accuracy: 0.8748
Epoch 13/30
273/273 [==============================] - 0s 696us/step - loss: 0.2848 - accu

In [20]:
print('word2vec: ',history['word2vec']/history['w_count'])
print('doc2vec  : ',history['doc2vec']/history['d_count'])

word2vec:  0.8157509157509157
doc2vec  :  0.823168498168498


In [21]:
accuracy, model, x_test, y_test = gakusyu(data_doc, labels_doc)

トレーニングデータの全体の数: 4368
トレーニングデータの1の数    : 2216
トレーニングデータ1の割合    : 50.7 %
テストデータの全体の数         : 1092
テストデータの１の数            : 514
テストデータの1の割合          : 47.1 %
Epoch 1/30
273/273 [==============================] - 0s 696us/step - loss: 0.5931 - accuracy: 0.6701
Epoch 2/30
273/273 [==============================] - 0s 662us/step - loss: 0.5361 - accuracy: 0.7250
Epoch 3/30
273/273 [==============================] - 0s 711us/step - loss: 0.5031 - accuracy: 0.7461
Epoch 4/30
273/273 [==============================] - 0s 686us/step - loss: 0.4730 - accuracy: 0.7747
Epoch 5/30
273/273 [==============================] - 0s 681us/step - loss: 0.4305 - accuracy: 0.7951
Epoch 6/30
273/273 [==============================] - 0s 725us/step - loss: 0.4062 - accuracy: 0.8148
Epoch 7/30
273/273 [==============================] - 0s 701us/step - loss: 0.3779 - accuracy: 0.8391
Epoch 8/30
273/273 [==============================] - 0s 676us/step - loss: 0.3614 - accuracy: 0.8379
Epoch 9/30
273/273 [========

In [22]:
y_test = np_utils.to_categorical(y_test)
pre = model.predict(x_test)
miss_count = 0
miss_count_1 = 0
miss_0 = []
miss_1 = []
for i,v in enumerate(pre):
    pre_ans = v.argmax()
    ans = y_test[i].argmax()
    dat = x_test[i]
    if ans == pre_ans: continue
    miss_count +=1
    if ans == 1:
        miss_count_1 += 1
        miss_1.append('関連があるが関連が無いと予測    :'+data_dict_doc[tuple(dat)])
    else:
        miss_0.append('関連が無いが関連があると予測    :'+data_dict_doc[tuple(dat)])

In [23]:
print('予測が失敗した数:',miss_count)
print('0の予測が失敗した数:',miss_count-miss_count_1)
print('1の予測が失敗した数:',miss_count_1)

予測が失敗した数: 179
0の予測が失敗した数: 119
1の予測が失敗した数: 60


In [24]:
miss_1

['関連があるが関連が無いと予測    :広島市西区スポーツセンター : ビーチ・プール',
 '関連があるが関連が無いと予測    :鞆の浦温泉 : 温泉・癒し・スパ',
 '関連があるが関連が無いと予測    :県央の森公園 : 紅葉名所',
 '関連があるが関連が無いと予測    :山根対厳堂 ギャラリー耀 : お土産・カフェ等',
 '関連があるが関連が無いと予測    :ＳＵＰ体験 : 紅葉名所',
 '関連があるが関連が無いと予測    :湯来ロッジ : 体験・見学',
 '関連があるが関連が無いと予測    :シーパーク大浜 : ビーチ・プール',
 '関連があるが関連が無いと予測    :帝釈峡 : クルーズ・船',
 '関連があるが関連が無いと予測    :広島市佐伯区スポーツセンター : ビーチ・プール',
 '関連があるが関連が無いと予測    :瀑雪の滝 : スポーツ施設',
 '関連があるが関連が無いと予測    :広島原爆被爆者療養研究センタークアハウス神田山荘 : 温泉・癒し・スパ',
 '関連があるが関連が無いと予測    :高茂温泉鵜の子荘(観光スポット) : 温泉・癒し・スパ',
 '関連があるが関連が無いと予測    :【安芸太田町】三段峡 : 体験・見学',
 '関連があるが関連が無いと予測    :東風崎神社 : 桜名所',
 '関連があるが関連が無いと予測    :仙石庭園 : 美術館・博物館',
 '関連があるが関連が無いと予測    :休暇村大久野島 : お土産・カフェ等',
 '関連があるが関連が無いと予測    :宮島海景散策 毛利丸 : クルーズ・船',
 '関連があるが関連が無いと予測    :レイクパーク本庄 : フラワー',
 '関連があるが関連が無いと予測    :ハイヅカ湖畔の森 : フラワー',
 '関連があるが関連が無いと予測    :グリーンポート吾妻路 : お土産・カフェ等',
 '関連があるが関連が無いと予測    :工場夜景 : 体験・見学',
 '関連があるが関連が無いと予測    :果実の森公園 : 体験・見学',
 '関連があるが関連が無いと予測    :ホットカモ : 温泉・癒し・スパ',
 '関連があるが関連が無いと予測    :道の駅伯方Ｓ・Ｃパーク マ

In [25]:
miss_0

['関連が無いが関連があると予測    :法王ヶ原 : 桜名所',
 '関連が無いが関連があると予測    :独立行政法人酒類総合研究所 : 自然・景勝地',
 '関連が無いが関連があると予測    :因島大橋記念公園 : 安全安心対策',
 '関連が無いが関連があると予測    :たけはら海の駅 : 道の駅',
 '関連が無いが関連があると予測    :さくらの里 : 公園・キャンプ場',
 '関連が無いが関連があると予測    :広島県立総合体育館(グリーンアリーナ) : 自然・景勝地',
 '関連が無いが関連があると予測    :【北広島町】長沢の枝垂れ桜 : フラワー',
 '関連が無いが関連があると予測    :鯛料理 : 安全安心対策',
 '関連が無いが関連があると予測    :広島市観光案内所(平和記念公園内)(ビジットジャパン案内所) : フラワー',
 '関連が無いが関連があると予測    :【北広島町】大丸峯 : 体験・見学',
 '関連が無いが関連があると予測    :長島大橋 : 安全安心対策',
 '関連が無いが関連があると予測    :国民宿舎みやじま杜の宿 : 体験・見学',
 '関連が無いが関連があると予測    :木江ふれあい郷土資料館 : 美術館・博物館',
 '関連が無いが関連があると予測    :瀬戸内オーシャンスパ 汐音 : お土産・カフェ等',
 '関連が無いが関連があると予測    :広島県立図書館 : テーマパーク',
 '関連が無いが関連があると予測    :広島市水産振興センター(魚と漁業の資料展示室) : 安全安心対策',
 '関連が無いが関連があると予測    :弥栄オートキャンプ場 : 安全安心対策',
 '関連が無いが関連があると予測    :呉カントリークラブ : 安全安心対策',
 '関連が無いが関連があると予測    :県立中央森林公園 : 桜名所',
 '関連が無いが関連があると予測    :中国醸造直営ショップ「SAKURAO蒸留所ビジターセンター」(SAKURAO DISTILLERY VISITOR CENTER) : フラワー',
 '関連が無いが関連があると予測    :安国寺釈迦堂 : 自然・景勝地',
 '関連が無いが関連があると予測    :竹炭 : 温泉・癒し・スパ',
 

In [26]:
count_1 = sum([i.argmax() for i in y_test]) #テストデータの１の数
print('全体の正解率:',1-miss_count/x_test.shape[0])
print('0の正解率    :',1-(miss_count - miss_count_1)/(x_test.shape[0]-count_1))
print('1の正解率    :',1-miss_count_1/count_1)

全体の正解率: 0.8360805860805861
0の正解率    : 0.7941176470588236
1の正解率    : 0.8832684824902723


In [27]:
accuracy, m_word, x_test, y_test = gakusyu(data_word, labels_word)
accuracy, m_doc, x_test, y_test = gakusyu(data_doc, labels_doc)

トレーニングデータの全体の数: 4368
トレーニングデータの1の数    : 2204
トレーニングデータ1の割合    : 50.5 %
テストデータの全体の数         : 1092
テストデータの１の数            : 526
テストデータの1の割合          : 48.2 %
Epoch 1/30
273/273 [==============================] - 0s 648us/step - loss: 0.5694 - accuracy: 0.7228
Epoch 2/30
273/273 [==============================] - 0s 643us/step - loss: 0.4622 - accuracy: 0.7811
Epoch 3/30
273/273 [==============================] - 0s 623us/step - loss: 0.4172 - accuracy: 0.8127
Epoch 4/30
273/273 [==============================] - 0s 610us/step - loss: 0.3885 - accuracy: 0.8292
Epoch 5/30
273/273 [==============================] - 0s 615us/step - loss: 0.3677 - accuracy: 0.8288
Epoch 6/30
273/273 [==============================] - 0s 619us/step - loss: 0.3521 - accuracy: 0.8427
Epoch 7/30
273/273 [==============================] - 0s 620us/step - loss: 0.3399 - accuracy: 0.8446
Epoch 8/30
273/273 [==============================] - 0s 624us/step - loss: 0.3222 - accuracy: 0.8567
Epoch 9/30
273/273 [========

In [28]:
land_dic_word = {}
for index, row in df.iterrows():
    land_dic_word[row['観光名所']] = weighted_mean_vec(row['観光名所'])

In [29]:
land_dic_doc = {}
for index, row in df.iterrows():
    land_dic_doc[row['観光名所']] = model_doc.infer_vector(get_tags_for_doc2vec(row['観光名所']))

In [30]:
def system(text, land_dic, model, model_type = 'word'):
    result = []
    data = []
    data_name = []
    if model_type == 'word':
        x2 = weighted_mean_vec(text)
    else:
        x2 = model_doc.infer_vector(get_tags_for_doc2vec(text))
    for k,v in land_dic.items():
        x1 = v
        data.extend([np.append(x1, x2)])
        data_name.append(k)
    data = np.array(data)
    pre = model.predict(data)
    for i,v in enumerate(pre):
        result.append(v[1])
    result_index = [i[0] for i in sorted(enumerate(result), key=lambda x:x[1],reverse=True)]
    result = []
    for i in result_index[0:10]:
        result.append(data_name[i])

    for i in result:
        for index,row in df.iterrows():
            if row['観光名所'] == i:
                print('{: <30}'.format(row['観光名所']),end=':')
                for i in df:
                    if i == '観光名所':
                        continue
                    else:
                        if row[i] == 1:
                            print(i,end='   ')
        print('') 

In [31]:
#word2vec
system('温泉', land_dic_word, m_word)

比婆山温泉熊野湯                      :温泉・癒し・スパ   
スーパー銭湯天然温泉昭和温泉郷湯楽里            :安全安心対策   温泉・癒し・スパ   
光信寺の湯ゆっくら                     :スポーツ施設   安全安心対策   温泉・癒し・スパ   
比和温泉あけぼの荘                     :温泉・癒し・スパ   
潮原温泉                          :安全安心対策   温泉・癒し・スパ   
温泉《賀茂川荘》                      :温泉・癒し・スパ   
宮浜温泉 湯の宿 宮浜グランドホテル            :安全安心対策   クルーズ・船   温泉・癒し・スパ   
女鹿平温泉めがひらスキー場                 :スポーツ施設   美術館・博物館   安全安心対策   スキー   
女鹿平温泉                         :安全安心対策   温泉・癒し・スパ   
半べえ温泉                         :温泉・癒し・スパ   


In [32]:
#word2vec
system('温泉に行きたい', land_dic_word, m_word)

矢野かもじ                         :お土産・カフェ等   
ロマンチックビーチかるが　※大雨災害の影響で営業再開未定※ :安全安心対策   ビーチ・プール   
広島県緑化センター・県立広島緑化植物公園          :桜名所   安全安心対策   公園・キャンプ場   紅葉名所   
県立もみのき森林公園                    :スポーツ施設   自然・景勝地   サイクリング   安全安心対策   紅葉名所   スキー   
広島県緑化センター・県立広島緑化植物公園(桜)       :桜名所   
たけはら海の駅                       :安全安心対策   お土産・カフェ等   クルーズ・船   
大芝公園「交通ランド」                   :自然・景勝地   
ONOMICHI U2                   :サイクリング   安全安心対策   お土産・カフェ等   
スノーフィールド県立もみのき森林公園            :スポーツ施設   安全安心対策   スキー   
ウイングパークゴルフ                    :桜名所   スポーツ施設   フラワー   


In [33]:
#doc2vec
system('温泉', land_dic_doc, m_doc, model_type = 'doc')

きのえ温泉ホテル清風館【宿泊】               :体験・見学   自然・景勝地   安全安心対策   温泉・癒し・スパ   
県民の浜 輝きの館・コテージかまがり【宿泊】        :ビーチ・プール   温泉・癒し・スパ   
女鹿平温泉クヴェーレ吉和【宿泊】              :安全安心対策   スキー   温泉・癒し・スパ   
広島エアポートホテル【宿泊】                :スポーツ施設   サイクリング   ビーチ・プール   公園・キャンプ場   
道の駅 ふぉレスト君田                   :自然・景勝地   道の駅   安全安心対策   
真名板海岸　※状況不明　売店自販機なし※          :ビーチ・プール   
天然温泉くらはし桂浜温泉館                 :安全安心対策   温泉・癒し・スパ   
瀬戸内しまなみ海道サイクルきっぷ発売！           :スポーツ施設   自然・景勝地   
【北広島町】道の駅 豊平どんぐり村             :体験・見学   自然・景勝地   道の駅   安全安心対策   ビーチ・プール   
【北広島町】やわたハイランド191リゾート         :スポーツ施設   安全安心対策   スキー   


In [35]:
#doc2vec
system('温泉に行きたい', land_dic_doc, m_doc, model_type = 'doc')

とびしま海道 下蒲刈島 「三之瀬エリア・文化施設散策 最高のフォトジェニック」:体験・見学   自然・景勝地   美術館・博物館   安全安心対策   
Kitchen’s シーサイド・はつかいち食堂       :安全安心対策   お土産・カフェ等   
瀬戸内しまなみ海道サイクルきっぷ発売！           :スポーツ施設   自然・景勝地   
県民の浜 輝きの館・コテージかまがり【宿泊】        :ビーチ・プール   温泉・癒し・スパ   
きのえ温泉ホテル清風館【宿泊】               :体験・見学   自然・景勝地   安全安心対策   温泉・癒し・スパ   
真名板海岸　※状況不明　売店自販機なし※          :ビーチ・プール   
瀬戸内しまなみ海道・宮窪瀬戸 潮流体験           :体験・見学   自然・景勝地   安全安心対策   クルーズ・船   
広島エアポートホテル【宿泊】                :スポーツ施設   サイクリング   ビーチ・プール   公園・キャンプ場   
Jカフェ 福山                       :安全安心対策   
とことこトレイン                      :体験・見学   安全安心対策   


In [36]:
#word2vec
system('世界遺産', land_dic_word, m_word)

嚴島神社宝物館                       :美術館・博物館   安全安心対策   世界遺産   
ウッドワン美術館                      :美術館・博物館   安全安心対策   
泉美術館                          :美術館・博物館   
東広島市立美術館                      :美術館・博物館   
はらみちを美術館                      :美術館・博物館   安全安心対策   
三良坂平和美術館                      :美術館・博物館   安全安心対策   
かぐや姫美術館                       :美術館・博物館   安全安心対策   
広島世界遺産定期観光バス                  :安全安心対策   世界遺産   
紙ヒコーキ博物館                      :美術館・博物館   
海田町ふるさと館（織田幹雄関連資料展示）          :安全安心対策   


In [37]:
#word2vec
system('海上に浮かんでいるように建つ嚴島神社は、原始宗教のなごりで、島全体が神の島として崇められており、陸地では畏れ多いと海中に社が建てられています', land_dic_word, m_word)


ロマンチックビーチかるが　※大雨災害の影響で営業再開未定※ :安全安心対策   ビーチ・プール   
矢野かもじ                         :お土産・カフェ等   
たけはら海の駅                       :安全安心対策   お土産・カフェ等   クルーズ・船   
広島県緑化センター・県立広島緑化植物公園          :桜名所   安全安心対策   公園・キャンプ場   紅葉名所   
のっとこクルーズ                      :体験・見学   安全安心対策   クルーズ・船   
広島県緑化センター・県立広島緑化植物公園(桜)       :桜名所   
ＳＵＰ体験                         :体験・見学   安全安心対策   紅葉名所   
広島城遊覧船運航                      :体験・見学   安全安心対策   
【現在休止中】来島海峡大橋登頂体験ツアー          :体験・見学   
ウイングパークゴルフ                    :桜名所   スポーツ施設   フラワー   


In [38]:
#doc2vec
system('世界遺産', land_dic_doc, m_doc, model_type = 'doc')

とびしま海道 下蒲刈島 「三之瀬エリア・文化施設散策 最高のフォトジェニック」:体験・見学   自然・景勝地   美術館・博物館   安全安心対策   
神勝寺 禅と庭のミュージアム                :体験・見学   美術館・博物館   安全安心対策   お土産・カフェ等   紅葉名所   
HAKKOパーク                      :テーマパーク   安全安心対策   
帝釈峡遊覧船【帝釈峡遊覧船】                :体験・見学   自然・景勝地   安全安心対策   紅葉名所   
耕三寺博物館（耕三寺）                   :桜名所   美術館・博物館   安全安心対策   紅葉名所   
とびしま海道　サイクリングロード              :サイクリング   
瀬戸内しまなみ海道・宮窪瀬戸 潮流体験           :体験・見学   自然・景勝地   安全安心対策   クルーズ・船   
真名板海岸　※状況不明　売店自販機なし※          :ビーチ・プール   
とことこトレイン                      :体験・見学   安全安心対策   
道の駅 さんわ182ステーション              :道の駅   安全安心対策   お土産・カフェ等   


In [39]:
#doc2vec
system('海上に浮かんでいるように建つ嚴島神社は、原始宗教のなごりで、島全体が神の島として崇められており、陸地では畏れ多いと海中に社が建てられています', land_dic_doc, m_doc, model_type = 'doc')

吾妻子の滝(東子の滝)                   :自然・景勝地   
広島県緑化センター・県立広島緑化植物公園(桜)       :桜名所   
弁天島(名勝鞆公園内)                   :自然・景勝地   
【北広島町】仙水湖(王泊ダム)               :自然・景勝地   紅葉名所   
瀬戸の絶景　大平山公園（桜名所・花見・展望）        :桜名所   自然・景勝地   公園・キャンプ場   
錦帯橋・吉香公園(桜)                   :桜名所   
羽高湖サン・スポーツランド&森林公園            :桜名所   自然・景勝地   安全安心対策   公園・キャンプ場   
帝釈峡遊覧船【帝釈峡遊覧船】                :体験・見学   自然・景勝地   安全安心対策   紅葉名所   
弥栄湖(弥栄ダム)                     :自然・景勝地   公園・キャンプ場   
グリーンヒル郷原(観光スポット)              :体験・見学   安全安心対策   フラワー   


In [40]:
#word2vec
system('スキー', land_dic_word, m_word)

りんご今日話国スキー場                   :スポーツ施設   スキー   
スーパー銭湯ふかわの湯                   :安全安心対策   温泉・癒し・スパ   
夢ヶ丘温泉湯めガーデン                   :温泉・癒し・スパ   
光信寺の湯ゆっくら                     :スポーツ施設   安全安心対策   温泉・癒し・スパ   
女鹿平温泉めがひらスキー場                 :スポーツ施設   美術館・博物館   安全安心対策   スキー   
女鹿平温泉                         :安全安心対策   温泉・癒し・スパ   
民宿 あるぺん屋【宿泊】                  :体験・見学   スキー   
神石高原ホテル                       :自然・景勝地   安全安心対策   
宮浜温泉 湯の宿 宮浜グランドホテル            :安全安心対策   クルーズ・船   温泉・癒し・スパ   
ロマンチックビーチかるが　※大雨災害の影響で営業再開未定※ :安全安心対策   ビーチ・プール   


In [41]:
#word2vec
system('美味しいご飯を食べたい', land_dic_word, m_word)

りんご今日話国スキー場                   :スポーツ施設   スキー   
農水産物加工品                       :お土産・カフェ等   
ホットカモ                         :温泉・癒し・スパ   
新鮮野菜市                         :お土産・カフェ等   
貝と海藻の家                        :美術館・博物館   安全安心対策   
矢野かもじ                         :お土産・カフェ等   
ロマンチックビーチかるが　※大雨災害の影響で営業再開未定※ :安全安心対策   ビーチ・プール   
陶芸体験《ぽれぽれ工房》                  :体験・見学   安全安心対策   
のっとこクルーズ                      :体験・見学   安全安心対策   クルーズ・船   
ネロリの島cafe                     :お土産・カフェ等   


In [43]:
#doc2vec
system('スキー', land_dic_doc, m_doc, model_type = 'doc')

女鹿平温泉クヴェーレ吉和【宿泊】              :安全安心対策   スキー   温泉・癒し・スパ   
瀬戸内しまなみ海道サイクルきっぷ発売！           :スポーツ施設   自然・景勝地   
広島エアポートホテル【宿泊】                :スポーツ施設   サイクリング   ビーチ・プール   公園・キャンプ場   
真名板海岸　※状況不明　売店自販機なし※          :ビーチ・プール   
きのえ温泉ホテル清風館【宿泊】               :体験・見学   自然・景勝地   安全安心対策   温泉・癒し・スパ   
芸北オークガーデン【宿泊】                 :スポーツ施設   体験・見学   安全安心対策   温泉・癒し・スパ   
梅の里《閉鎖中につき入園不可》               :ビーチ・プール   フラワー   
サイクリング《竹原観光案内処》               :体験・見学   安全安心対策   
【北広島町】やわたハイランド191リゾート         :スポーツ施設   安全安心対策   スキー   
県民の浜 輝きの館・コテージかまがり【宿泊】        :ビーチ・プール   温泉・癒し・スパ   


In [44]:
#doc2vec
system('美味しいご飯を食べたい', land_dic_doc, m_doc, model_type = 'doc')

Kitchen’s シーサイド・はつかいち食堂       :安全安心対策   お土産・カフェ等   
梶ヶ浜海水浴場(コテージ梶ヶ浜)              :安全安心対策   ビーチ・プール   公園・キャンプ場   
Hamburger & TacoRice O2       :安全安心対策   お土産・カフェ等   
ジャム作り体験、 工場見学《アヲハタ ジャムデッキ》    :体験・見学   安全安心対策   
県民の浜 輝きの館・コテージかまがり【宿泊】        :ビーチ・プール   温泉・癒し・スパ   
ゆず狩り亀の丸果樹生産組合                 :体験・見学   フラワー   
大竹手打刃物※周辺県道整備に伴い,現在は見学不可      :体験・見学   
山根対厳堂 ギャラリー耀                  :体験・見学   安全安心対策   お土産・カフェ等   
せとうち古民家ステイズHiroshima          :体験・見学   
着物体験《竹楽》                      :体験・見学   安全安心対策   お土産・カフェ等   


In [45]:
#word2vec
system('花見をしたい', land_dic_word, m_word)

矢野かもじ                         :お土産・カフェ等   
広島県緑化センター・県立広島緑化植物公園          :桜名所   安全安心対策   公園・キャンプ場   紅葉名所   
ロマンチックビーチかるが　※大雨災害の影響で営業再開未定※ :安全安心対策   ビーチ・プール   
たけはら海の駅                       :安全安心対策   お土産・カフェ等   クルーズ・船   
広島県緑化センター・県立広島緑化植物公園(桜)       :桜名所   
ウイングパークゴルフ                    :桜名所   スポーツ施設   フラワー   
県立もみのき森林公園                    :スポーツ施設   自然・景勝地   サイクリング   安全安心対策   紅葉名所   スキー   
東広島市観光振興課公式LINE@アカウントを開設しました！ :観光案内所   
宮島海景散策 毛利丸                    :体験・見学   自然・景勝地   安全安心対策   クルーズ・船   
大芝公園「交通ランド」                   :自然・景勝地   


In [46]:
#doc2vec
system('花見をしたい', land_dic_doc, m_doc, model_type = 'doc')

Kitchen’s シーサイド・はつかいち食堂       :安全安心対策   お土産・カフェ等   
山根対厳堂 ギャラリー耀                  :体験・見学   安全安心対策   お土産・カフェ等   
とびしま海道 下蒲刈島 「三之瀬エリア・文化施設散策 最高のフォトジェニック」:体験・見学   自然・景勝地   美術館・博物館   安全安心対策   
県民の浜 輝きの館・コテージかまがり【宿泊】        :ビーチ・プール   温泉・癒し・スパ   
香淀の大イチョウ                      :紅葉名所   
広島大学キャンパスガイド(広島大学東広島キャンパス)    :美術館・博物館   
爽籟軒庭園                         :自然・景勝地   
神勝寺 禅と庭のミュージアム                :体験・見学   美術館・博物館   安全安心対策   お土産・カフェ等   紅葉名所   
瀬戸内しまなみ海道サイクルきっぷ発売！           :スポーツ施設   自然・景勝地   
福山市園芸センター                     :テーマパーク   安全安心対策   
